In [8]:
# -*- encoding: utf-8 -*-

from osgeo import gdal
from osgeo import osr
import numpy as np


def getSRSPair(dataset):
    """
    获得给定数据的投影参考系和地理参考系
    :param dataset: GDAL地理数据
    :return: 投影参考系和地理参考系
    """
    prosrs = osr.SpatialReference()
    prosrs.ImportFromWkt(dataset.GetProjection())
    geosrs = prosrs.CloneGeogCS()
    return prosrs, geosrs


def geo2lonlat(dataset, x, y):
    """
    将投影坐标转为经纬度坐标（具体的投影坐标系由给定数据确定）
    :param dataset: GDAL地理数据
    :param x: 投影坐标x
    :param y: 投影坐标y
    :return: 投影坐标(x, y)对应的经纬度坐标(lon, lat)
    """
    prosrs, geosrs = getSRSPair(dataset)
    ct = osr.CoordinateTransformation(prosrs, geosrs)
    coords = ct.TransformPoint(x, y)
    return coords[:2]


def lonlat2geo(dataset, lon, lat):
    """
    将经纬度坐标转为投影坐标（具体的投影坐标系由给定数据确定）
    :param dataset: GDAL地理数据
    :param lon: 地理坐标lon经度
    :param lat: 地理坐标lat纬度
    :return: 经纬度坐标(lon, lat)对应的投影坐标
    """
    prosrs, geosrs = getSRSPair(dataset)
    ct = osr.CoordinateTransformation(geosrs, prosrs)
    coords = ct.TransformPoint(lat, lon)
    return coords[:2]


def imagexy2geo(dataset, row, col):
    """
    根据GDAL的六参数模型将影像图上坐标（行列号）转为投影坐标或地理坐标（根据具体数据的坐标系统转换）
    :param dataset: GDAL地理数据
    :param row: 像素的行号
    :param col: 像素的列号
    :return: 行列号(row, col)对应的投影坐标或地理坐标(x, y)
    """
    trans = dataset.GetGeoTransform()
    px = trans[0] + col * trans[1] + row * trans[2]
    py = trans[3] + col * trans[4] + row * trans[5]
    return px, py


def geo2imagexy(dataset, x, y):
    """
    根据GDAL的六 参数模型将给定的投影或地理坐标转为影像图上坐标（行列号）
    :param dataset: GDAL地理数据
    :param x: 投影或地理坐标x
    :param y: 投影或地理坐标y
    :return: 影坐标或地理坐标(x, y)对应的影像图上行列号(row, col)
    """
    trans = dataset.GetGeoTransform()
    a = np.array([[trans[1], trans[2]], [trans[4], trans[5]]])
    b = np.array([x - trans[0], y - trans[3]])
    return np.linalg.solve(a, b)  # 使用numpy的linalg.solve进行二元一次方程的求解


if __name__ == "__main__":
    gdal.AllRegister()
    dataset = gdal.Open("./dem/贵阳/贵阳.tif")
    print("数据投影：")
    print(dataset.GetProjection())
    print("数据的大小（行，列）：")
    print("(%s %s)" % (dataset.RasterYSize, dataset.RasterXSize))

    x = 90428.6
    y = 3016604.2
    lon = 106.86622934
    lat = 27.21131557
    row = 6124
    col = 1271

    print("投影坐标 -> 经纬度：")
    coords = geo2lonlat(dataset, x, y)
    print("(%s, %s)->(%s, %s)" % (x, y, coords[0], coords[1]))
    print("经纬度 -> 投影坐标：")
    coords = lonlat2geo(dataset, lon, lat)
    print("(%s, %s)->(%s, %s)" % (lon, lat, coords[0], coords[1]))

    print("图上坐标 -> 投影坐标：")
    coords = imagexy2geo(dataset, row, col)
    print("(%s, %s)->(%s, %s)" % (row, col, coords[0], coords[1]))
    print("投影坐标 -> 图上坐标：")
    coords = geo2imagexy(dataset, x, y)
    print("(%s, %s)->(%s, %s)" % (x, y, coords[0], coords[1]))

数据投影：
PROJCS["WGS 84 / UTM zone 49N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",111],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32649"]]
数据的大小（行，列）：
(10381 9276)
投影坐标 -> 经纬度：
(90428.6, 3016604.2)->(27.21131240359315, 106.86623112217411)
经纬度 -> 投影坐标：
(106.86622934, 27.21131557)->(90428.43485249992, 3016604.5570959183)
图上坐标 -> 投影坐标：
(6124, 1271)->(29799.553394642087, 2955986.667934197)
投影坐标 -> 图上坐标：
(90428.6, 3016604.2)->(6124.5007224250185, 1271.424069777872)


In [9]:
from osgeo import osr, ogr


def convert_lat_lon_to_projected_coordinates(
    latitude, longitude, input_srs, output_srs
):
    # 创建坐标系统对象
    in_srs = osr.SpatialReference()
    in_srs.ImportFromEPSG(input_srs)  # 输入的EPSG码，例如WGS84的EPSG码是4326

    out_srs = osr.SpatialReference()
    out_srs.ImportFromEPSG(output_srs)  # 输出的EPSG码，例如UTM Zone 49N的EPSG码是32649

    # 创建坐标变换对象
    coord_transform = osr.CoordinateTransformation(in_srs, out_srs)

    # 创建一个点对象，并设置其坐标
    point = ogr.Geometry(ogr.wkbPoint)
    point.AddPoint(longitude, latitude)  # 注意顺序：经度在前，纬度在后

    # 应用坐标变换
    transformed_point = coord_transform.TransformPoint(longitude, latitude)

    # 检查变换结果是否有效
    if transformed_point[0] != transformed_point[0]:
        raise ValueError("Coordinate transformation failed.")

    return transformed_point[0], transformed_point[1]


# 示例使用
if __name__ == "__main__":
    # 给定的经纬度坐标
    lat = 106.86622934
    lon = 27.21131557

    # 输入坐标系统的EPSG码（WGS84）
    input_srs = 4326

    # 输出坐标系统的EPSG码（例如，UTM Zone 49N）
    output_srs = 32649

    try:
        # 转换坐标
        x, y = convert_lat_lon_to_projected_coordinates(lat, lon, input_srs, output_srs)
        print(f"Projected Coordinates: X={x}, Y={y}")
    except ValueError as e:
        print(e)

Projected Coordinates: X=90428.43485249992, Y=3016604.5570959183
